In [1]:
import pandas as pd
import numpy as np
import anndata as ad

# Metadata

In [2]:
!ls ../data_dump/candidate_genes*

../data_dump/candidate_genes_vincent_0316.xlsx
../data_dump/candidate_genes_vincent_0328.csv
../data_dump/candidate_genes_vincent_0503.csv


In [27]:
f = '../data_dump/candidate_genes_vincent_0503.csv'
# df0 = pd.read_excel(f, sheet_name=0)
df0 = pd.read_csv(f)
print(df0.shape)

(88, 6)


In [28]:
df0 = df0.groupby('Gene').first().reset_index()
print(df0.shape)
df0 = df0[~df0['Gene'].isnull()]
print(df0.shape)
df0['Gene'] = df0['Gene'].apply(lambda x: x[0].upper() + x[1:].lower())
print(df0.shape)
df0

(84, 6)
(84, 6)
(84, 6)


,Gene,Subclass,Type/Gradient,Function,Priority,Allen marker
0,Adamts2,L2/3,A>B=C,other,***,x
1,Agmat,L2/3,None,other,None,x
2,Astn2,L2/3,C>B>A,Syn,None,None
3,Baz1a,L2/3,A<B>C,TF,***,None
4,Bdnf,L2/3,A<B>C,CAM,***,None
...,...,...,...,...,...,...
79,Tbr1,L2/3,C>B>A,TF,None,None
80,Tenm1,L2/3,A>B>C,CAM,None,None
81,Trpc6,L2/3,A<B>C,IC,***,None
82,Whrn,L4,None,other,None,None


# Files 

In [29]:
f = '../data_dump/P38_glut.h5ad'
adata = ad.read(f)
adata

AnnData object with n_obs × n_vars = 10485 × 30869
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Type_colors', 'cluster_colors', "dendrogram_['leiden']", 'hvg', 'layer_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

# save specific genes

In [30]:
cgenes = np.intersect1d(adata.var.index.values, df0['Gene'].values)
print([g for g in df0['Gene'].values if g not in cgenes])
cgenes.shape

['Neurod1']


(83,)

In [31]:
df0 = df0[df0['Gene'].isin(cgenes)]
df0

,Gene,Subclass,Type/Gradient,Function,Priority,Allen marker
0,Adamts2,L2/3,A>B=C,other,***,x
1,Agmat,L2/3,None,other,None,x
2,Astn2,L2/3,C>B>A,Syn,None,None
3,Baz1a,L2/3,A<B>C,TF,***,None
4,Bdnf,L2/3,A<B>C,CAM,***,None
...,...,...,...,...,...,...
79,Tbr1,L2/3,C>B>A,TF,None,None
80,Tenm1,L2/3,A>B>C,CAM,None,None
81,Trpc6,L2/3,A<B>C,IC,***,None
82,Whrn,L4,None,other,None,None


In [32]:
# f = '../data_dump/candidate_genes_vincent_0503_v2.csv'
# df0.to_csv(f, header=True, index=False)

In [33]:
f = '../data_dump/candidate_genes_vincent_0503_v2.csv'
df0 = pd.read_csv(f)
df0

,Gene,Subclass,Type/Gradient,Function,Priority,Allen marker
0,Adamts2,L2/3,A>B=C,other,***,x
1,Agmat,L2/3,NaN,other,NaN,x
2,Astn2,L2/3,C>B>A,Syn,NaN,NaN
3,Baz1a,L2/3,A<B>C,TF,***,NaN
4,Bdnf,L2/3,A<B>C,CAM,***,NaN
...,...,...,...,...,...,...
78,Tbr1,L2/3,C>B>A,TF,NaN,NaN
79,Tenm1,L2/3,A>B>C,CAM,NaN,NaN
80,Trpc6,L2/3,A<B>C,IC,***,NaN
81,Whrn,L4,NaN,other,NaN,NaN


In [34]:
%%time
adata = adata[:,cgenes]
adata

CPU times: user 83.3 ms, sys: 571 ms, total: 654 ms
Wall time: 653 ms


View of AnnData object with n_obs × n_vars = 10485 × 83
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Type_colors', 'cluster_colors', "dendrogram_['leiden']", 'hvg', 'layer_colors', 'leiden', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [35]:
def rename_clsts(x):
    thedict = {
        'L2/3_A': 'L2/3_B',
        'L2/3_B': 'L2/3_A',
    }
    if x in thedict.keys():
        return thedict[x]
    else:
        return x

In [36]:
%%time
adata = adata[adata.obs['cluster'].str.contains(r'^L2|L4')]
adata.obs['cluster'] = adata.obs['cluster'].apply(rename_clsts) 
adata

Trying to set attribute `.obs` of view, copying.


CPU times: user 80.3 ms, sys: 720 ms, total: 800 ms
Wall time: 797 ms


AnnData object with n_obs × n_vars = 5837 × 83
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Type_colors', 'cluster_colors', "dendrogram_['leiden']", 'hvg', 'layer_colors', 'leiden', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_harmony', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [37]:
adata.write('/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/P38_glut_genes_v3.h5ad')